### Initializing the pyspark session and reading pyspark session

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("FirstSaprkSession").getOrCreate())

In [3]:
final_sales_df = spark.read.format('parquet')\
                    	.option("header", True)\
                    	.load('parquet_output/')

In [4]:
final_sales_df.show(5)

+-------+--------------------+--------+------+-------------------+--------------------+--------------+----------+-----+
|OrderID|             Product|Quantity| Price|          OrderDate|        StoreAddress|          City|ReportYear|Month|
+-------+--------------------+--------+------+-------------------+--------------------+--------------+----------+-----+
| 295665|  Macbook Pro Laptop|       1|1700.0|2019-12-30 00:01:00|[136 Church St,  ...| New York City|      2019|   12|
| 295666|  LG Washing Machine|       1| 600.0|2019-12-29 07:03:00|[562 2nd St,  New...| New York City|      2019|   12|
| 295667|USB-C Charging Cable|       1| 11.95|2019-12-12 18:21:00|[277 Main St,  Ne...| New York City|      2019|   12|
| 295668|    27in FHD Monitor|       1|149.99|2019-12-22 15:13:00|[410 6th St,  San...| San Francisco|      2019|   12|
| 295669|USB-C Charging Cable|       1| 11.95|2019-12-18 12:38:00|[43 Hill St,  Atl...|       Atlanta|      2019|   12|
+-------+--------------------+--------+-

### What was the best month in terms of sales?

In [12]:
from pyspark.sql.functions import col,expr,sum

In [7]:
best_month_df = final_sales_df.withColumn("total_sales", expr("Quantity * Price"))

In [8]:
best_month_df.show(5)

+-------+--------------------+--------+------+-------------------+--------------------+--------------+----------+-----+-----------+
|OrderID|             Product|Quantity| Price|          OrderDate|        StoreAddress|          City|ReportYear|Month|total_sales|
+-------+--------------------+--------+------+-------------------+--------------------+--------------+----------+-----+-----------+
| 295665|  Macbook Pro Laptop|       1|1700.0|2019-12-30 00:01:00|[136 Church St,  ...| New York City|      2019|   12|     1700.0|
| 295666|  LG Washing Machine|       1| 600.0|2019-12-29 07:03:00|[562 2nd St,  New...| New York City|      2019|   12|      600.0|
| 295667|USB-C Charging Cable|       1| 11.95|2019-12-12 18:21:00|[277 Main St,  Ne...| New York City|      2019|   12|      11.95|
| 295668|    27in FHD Monitor|       1|149.99|2019-12-22 15:13:00|[410 6th St,  San...| San Francisco|      2019|   12|     149.99|
| 295669|USB-C Charging Cable|       1| 11.95|2019-12-18 12:38:00|[43 Hill S

In [13]:
best_month_df = best_month_df.groupBy('Month').agg(sum("total_sales").alias("month_sales")).orderBy(col("month_sales"), ascending=False)

December is the best month for sales.

In [15]:
best_month_df.show(10,False)

+-----+------------------+
|Month|month_sales       |
+-----+------------------+
|12   |4613443.31607008  |
|10   |3736726.860444069 |
|4    |3390670.2231714725|
|11   |3199603.184257984 |
|5    |3152606.7349271774|
|3    |2807100.3656582832|
|7    |2647775.7468385696|
|6    |2577802.247295618 |
|8    |2244467.8685896397|
|2    |2202022.408319235 |
+-----+------------------+
only showing top 10 rows



### which city actually sold the most products?

In [17]:
best_city_df = final_sales_df.groupBy('City').agg(sum("Quantity").alias("city_products")).orderBy(col("city_products"), ascending=False)

San Francisco sold the most number of products.

In [18]:
best_city_df.show(10)

+--------------+-------------+
|          City|city_products|
+--------------+-------------+
| San Francisco|        50239|
|   Los Angeles|        33289|
| New York City|        27932|
|        Boston|        22528|
|        Dallas|        16730|
|       Atlanta|        16602|
|       Seattle|        16553|
|      Portland|        14053|
|        Austin|        11153|
+--------------+-------------+



### At what time should we display the advertisements to maximise the sales?

In [19]:
from pyspark.sql.functions import hour

First we extracted the hour from OrderDate column and then calculating total sales.

In [23]:
best_time_df = final_sales_df.withColumn("best_hour", hour(col("orderDate"))).withColumn("total_sales", expr("Quantity * Price"))

In [24]:
best_time_df.show(10,False)

+-------+--------------------------+--------+------+-------------------+---------------------------------------------+--------------+----------+-----+---------+-----------+
|OrderID|Product                   |Quantity|Price |OrderDate          |StoreAddress                                 |City          |ReportYear|Month|best_hour|total_sales|
+-------+--------------------------+--------+------+-------------------+---------------------------------------------+--------------+----------+-----+---------+-----------+
|295665 |Macbook Pro Laptop        |1       |1700.0|2019-12-30 00:01:00|[136 Church St,  New York City,  NY 10001]   | New York City|2019      |12   |0        |1700.0     |
|295666 |LG Washing Machine        |1       |600.0 |2019-12-29 07:03:00|[562 2nd St,  New York City,  NY 10001]      | New York City|2019      |12   |7        |600.0      |
|295667 |USB-C Charging Cable      |1       |11.95 |2019-12-12 18:21:00|[277 Main St,  New York City,  NY 10001]     | New York City|20

In [25]:
best_time_df = best_time_df.groupBy('best_hour').agg(sum("total_sales").alias("hour_sales")).orderBy(col("hour_sales"), ascending=False)

19th Hour(7 p.m.) is the best time for advertisements.

In [26]:
best_time_df.show(10,False)

+---------+------------------+
|best_hour|hour_sales        |
+---------+------------------+
|19       |2412938.5279734135|
|12       |2316821.328401327 |
|11       |2300610.2287039757|
|20       |2281716.2286674976|
|18       |2219348.288781166 |
|13       |2155389.7889335155|
|17       |2129361.5990903378|
|14       |2083672.7193861008|
|21       |2042000.8498032093|
|10       |1944286.759740591 |
+---------+------------------+
only showing top 10 rows



### What products are often sold together in NY?

In [44]:
ny_city_df = final_sales_df.filter(col("City")==' New York City')

In [45]:
ny_city_df.show(10,False)

+-------+--------------------------+--------+------+-------------------+---------------------------------------------+--------------+----------+-----+
|OrderID|Product                   |Quantity|Price |OrderDate          |StoreAddress                                 |City          |ReportYear|Month|
+-------+--------------------------+--------+------+-------------------+---------------------------------------------+--------------+----------+-----+
|295665 |Macbook Pro Laptop        |1       |1700.0|2019-12-30 00:01:00|[136 Church St,  New York City,  NY 10001]   | New York City|2019      |12   |
|295666 |LG Washing Machine        |1       |600.0 |2019-12-29 07:03:00|[562 2nd St,  New York City,  NY 10001]      | New York City|2019      |12   |
|295667 |USB-C Charging Cable      |1       |11.95 |2019-12-12 18:21:00|[277 Main St,  New York City,  NY 10001]     | New York City|2019      |12   |
|295670 |AA Batteries (4-pack)     |1       |3.84  |2019-12-31 22:58:00|[200 Jefferson St,  Ne

checking for records. Does OrderID multiple values are avaible. We found multiple values.

In [46]:
ny_city_df.groupBy("OrderID").count().show(5)

+-------+-----+
|OrderID|count|
+-------+-----+
| 296288|    1|
| 300522|    1|
| 300574|    1|
| 301535|    1|
| 304276|    2|
+-------+-----+
only showing top 5 rows



Using the collect_list() function to create list of all products.

In [51]:
from pyspark.sql.functions import concat_ws, collect_list, size

In [48]:
ny_city_df = ny_city_df.groupBy("OrderID").agg(collect_list(col("Product")).alias("OrderId_Products"))

In [50]:
ny_city_df.show(10, truncate=False)

+-------+----------------------------+
|OrderID|OrderId_Products            |
+-------+----------------------------+
|141264 |[Apple Airpods Headphones]  |
|141268 |[AA Batteries (4-pack)]     |
|141272 |[AAA Batteries (4-pack)]    |
|141278 |[Lightning Charging Cable]  |
|141282 |[Vareebadd Phone]           |
|141301 |[Apple Airpods Headphones]  |
|141314 |[Macbook Pro Laptop]        |
|141317 |[27in FHD Monitor]          |
|141323 |[USB-C Charging Cable]      |
|141343 |[Bose SoundSport Headphones]|
+-------+----------------------------+
only showing top 10 rows



In [52]:
ny_city_df = ny_city_df.withColumn("product_list_size", size("OrderId_Products"))

Final Dataframe sorted by product_list_size

In [55]:
ny_city_df.orderBy("product_list_size", ascending=False).show(10, False)

+-------+----------------------------------------------------------------+-----------------+
|OrderID|OrderId_Products                                                |product_list_size|
+-------+----------------------------------------------------------------+-----------------+
|144819 |[iPhone, Lightning Charging Cable, Wired Headphones]            |3                |
|182908 |[Google Phone, USB-C Charging Cable, Wired Headphones]          |3                |
|149499 |[Vareebadd Phone, USB-C Charging Cable, Wired Headphones]       |3                |
|149936 |[Google Phone, USB-C Charging Cable, Bose SoundSport Headphones]|3                |
|150951 |[iPhone, Lightning Charging Cable, Apple Airpods Headphones]    |3                |
|155053 |[Google Phone, Bose SoundSport Headphones, Wired Headphones]    |3                |
|157344 |[Vareebadd Phone, USB-C Charging Cable, Wired Headphones]       |3                |
|159000 |[iPhone, Lightning Charging Cable, Wired Headphones]         

In [57]:
ny_city_df = ny_city_df.filter(col("product_list_size") >1)

In [58]:
ny_city_df.show(10, truncate=False)

+-------+---------------------------------------------+-----------------+
|OrderID|OrderId_Products                             |product_list_size|
+-------+---------------------------------------------+-----------------+
|141365 |[Vareebadd Phone, Wired Headphones]          |2                |
|141645 |[Lightning Charging Cable, Wired Headphones] |2                |
|141910 |[Vareebadd Phone, Wired Headphones]          |2                |
|142039 |[Google Phone, USB-C Charging Cable]         |2                |
|142144 |[Vareebadd Phone, USB-C Charging Cable]      |2                |
|142320 |[Google Phone, iPhone]                       |2                |
|142389 |[iPhone, Lightning Charging Cable]           |2                |
|142419 |[Google Phone, Wired Headphones]             |2                |
|142651 |[USB-C Charging Cable, AA Batteries (4-pack)]|2                |
|142730 |[Google Phone, USB-C Charging Cable]         |2                |
+-------+-----------------------------

Group by the list of products that are sold together.

In [60]:
ny_city_df.groupBy("OrderId_Products").count().orderBy("count", ascending=False).show(10,False)

+------------------------------------------------------+-----+
|OrderId_Products                                      |count|
+------------------------------------------------------+-----+
|[iPhone, Lightning Charging Cable]                    |127  |
|[Google Phone, USB-C Charging Cable]                  |124  |
|[Google Phone, Wired Headphones]                      |52   |
|[Vareebadd Phone, USB-C Charging Cable]               |49   |
|[iPhone, Wired Headphones]                            |46   |
|[iPhone, Apple Airpods Headphones]                    |43   |
|[Google Phone, Bose SoundSport Headphones]            |23   |
|[Vareebadd Phone, Wired Headphones]                   |17   |
|[Apple Airpods Headphones, Wired Headphones]          |12   |
|[Google Phone, USB-C Charging Cable, Wired Headphones]|11   |
+------------------------------------------------------+-----+
only showing top 10 rows

